# recursion in F#

The [Microsoft documentation](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/functions/recursive-functions-the-rec-keyword) on the `rec` keyword begins with the F# rendition of the naïve Fibonacci algorithm:

In [ ]:
#!fsharp

let rec fib n =
    printfn $"before: {nameof n}: {n}"
    let result =
        match n with
        | 0 | 1 -> n // base case
        | n -> fib (n-1) + fib (n-2) // recursive case: this tail-call is augementing
    printfn $"after: {nameof n}: {n}"
    result

[0..4] |> List.map fib |> ignore

before: n: 0
after: n: 0
before: n: 1
after: n: 1
before: n: 2
before: n: 1
after: n: 1
before: n: 0
after: n: 0
after: n: 2
before: n: 3
before: n: 2
before: n: 1
after: n: 1
before: n: 0
after: n: 0
after: n: 2
before: n: 1
after: n: 1
after: n: 3
before: n: 4
before: n: 3
before: n: 2
before: n: 1
after: n: 1
before: n: 0
after: n: 0
after: n: 2
before: n: 1
after: n: 1
after: n: 3
before: n: 2
before: n: 1
after: n: 1
before: n: 0
after: n: 0
after: n: 2
after: n: 4


Notice that the naïve version takes over 5 seconds to print out the iterations over just _five_ numbers! Also notice that the `after:` numbers above are _repeated_. Microsoft chimes in:

>In practice, code like the previous sample is not ideal because it unecessarily recomputes values that have already been computed. This is because it is not tail recursive…

## tail recursion

Tail recursion [[StackExchange](https://cs.stackexchange.com/a/7814)] “prevents unnecessary recomputations”:

In [ ]:
#!fsharp

let fib n =
    printfn $"before: {nameof n}: {n}"

    let rec loop acc1 acc2 n =
        match n with
        | 0 -> acc1
        | 1 -> acc2
        | _ -> loop acc2 (acc1 + acc2) (n - 1) // tail recursion

    printfn $"after: {nameof n}: {n}"

    loop 0 1 n

[0..8] |> List.map fib

before: n: 0
after: n: 0
before: n: 1
after: n: 1
before: n: 2
after: n: 2
before: n: 3
after: n: 3
before: n: 4
after: n: 4
before: n: 5
after: n: 5
before: n: 6
after: n: 6
before: n: 7
after: n: 7
before: n: 8
after: n: 8


index,value
0,0
1,1
2,1
3,2
4,3
5,5
6,8
7,13
8,21


We see that the inner function `loop` calls itself on its last line (hence, _tail_ recursive). What is difficult to understand is how memoization (or accumulation) is handled by the elegant arrangement between `acc1` and `acc2`.

## memoization with a mutable dictionary

We can avoid the FP elegance above for the sake of understanding by sneaking in the CLR `Dictionary<K,V>` into F#:

In [ ]:
#!fsharp

open System.Collections.Generic
open System.Linq

let fib n =

    let cache = new Dictionary<int, int>()

    //mutate cache:
    cache.Add(0, 0)
    cache.Add(1, 1)
    cache.Add(2, 1)

    printfn $"before: {nameof n}: {n}"

    let rec loop n =
        if cache.Keys.Contains(n) then cache[n]
        else
            let v = loop(n - 1) + loop(n - 1)
            cache.Add(n, v) //mutate cache again
            v

    printfn $"after: {nameof n}: {n}"

    loop n

[0..8] |> List.map fib

before: n: 0
after: n: 0
before: n: 1
after: n: 1
before: n: 2
after: n: 2
before: n: 3
after: n: 3
before: n: 4
after: n: 4
before: n: 5
after: n: 5
before: n: 6
after: n: 6
before: n: 7
after: n: 7
before: n: 8
after: n: 8


index,value
0,0
1,1
2,1
3,2
4,4
5,8
6,16
7,32
8,64


[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
